In [26]:
import requests
import json
import pandas as pd
import numpy as np
import musicbrainzngs as mbz
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## Call MusicBrainz API

In [27]:
# return requests as xml
mbz.set_format('xml')

In [28]:
# set user agent
mbz.set_useragent(
    "ComputerFace-PureBeing",
    "0.1",
    "https://github.com/byronkking/ComputerFace-PureBeing/",
)

### get artist info

In [29]:
# find artist id for FlyLo
request = mbz.search_artists("Flying Lotus")
for artist in request['artist-list']:
    print(u"{id}: {name}".format(id=artist['id'], name=artist["name"]))

fc7376fe-1a6f-4414-b4a7-83f50ed59c92: Flying Lotus
b4681cdc-4002-4521-8458-ac812f1b6d28: Lotus
7f7a18ff-90b3-496e-a25c-4bbb34f51edf: Lotus
f15386df-5950-46ba-a899-ed74dfaf0a42: Lotus
467778bf-cee8-408a-b22a-fde6a99087ad: Lotus
88d3235a-6d73-4a17-a98d-93099e9bbbac: Lotus
28a6c75c-5d8f-483f-9c8c-829b89fb0050: Lotus
b2358a61-dd8f-47f4-a6ac-744c413c3fb0: Lotus
2891a20f-812f-4f8b-96a8-0b0361cd1785: Lotus
2e27c86e-fe90-434c-9d35-8fe2e07738fe: Lotus
b78b664d-e2ba-435f-a71a-180e73c9c767: Lotus
e3251e2b-0db2-47c4-b684-2efa6df0a00e: Lotus
59eaea88-3089-4c24-9ec9-5a7ae73936be: Lotus
cc5c45ad-b296-4e08-8ffd-9de7ad4ee298: Lotus
bc559041-304b-4ecf-833b-0cb2ad1a4ba6: Lotus
2cc924f5-d6d8-477d-be49-e77575d6b4df: Flying
1c5e627b-d16f-41c3-a031-48b956829343: Flying
58d0cb0e-a088-46e6-811b-c093cc77573d: Lotus Crown
3554dfec-821a-4d34-b707-0cac889ae81e: Dark Lotus
e60b0737-095c-4cff-8e35-f24063d7e212: Crimson Lotus
67776697-0ce5-476a-a763-b3d0548bad41: Lotus Project
b8ca92e7-3d72-4583-835b-3fefe4b8dcc9: Pu

In [30]:
# test getting FlyLo info with artist id
artist_id = "fc7376fe-1a6f-4414-b4a7-83f50ed59c92"
try:
    result = mbz.get_artist_by_id(artist_id)
except Exception as error:
    print("Something went wrong with the request: %s" % error)
else:
    artist = result["artist"]
    print("name:\t\t%s" % artist["name"])
    print("sort name:\t%s" % artist["sort-name"])

name:		Flying Lotus
sort name:	Flying Lotus


### get artist info

In [31]:
# test getting FlyLo info with artist id
artist_id = "fc7376fe-1a6f-4414-b4a7-83f50ed59c92"
result = mbz.get_artist_by_id(artist_id,includes=['releases'])

In [32]:
temp = pd.DataFrame.from_dict(result)

### get album info

#### albums and eps

In [33]:
result = mbz.get_artist_by_id(artist_id,includes=["release-groups","tags"], release_type=["album", "ep"])

In [35]:
for release_group in result["artist"]["release-group-list"]:
    print("({date}) {title} ({type})".format(date=release_group["first-release-date"],
                                            title=release_group["title"],type=release_group['type']))

(2015-07-04) Lovers Melt 4 (Album)
(2011-01-12) Cosmogramma Alt Takes (EP)
(2008-08) Shhh! (EP)
(2008-11-24) L.A. EP 2 X 3 (EP)
(2009) Tectonic Plate 2.3 (EP)
(2009-05-02) LA CD (Compilation)
(2010) IIOIO (Compilation)
(2009-02) Whole Wide World / Lit Up / Keep It Moving (EP)
(2008-06-04) Los Angeles (Album)
(2010-09-11) Pattern+Grid World (EP)
() 2009-04-05: Plastic People, London, UK (Live)
(2006-10-03) 1983 (Album)
(2014-10-06) You’re Dead! (Album)
(2007-10-01) Reset (EP)
() Raw Cartoons (Compilation)
(2008-06-30) L.A. EP 1 X 3 (EP)
(2009-08-17) L.A. EP 3 X 3 (EP)
(2010-04-21) Cosmogramma (Album)
(2012-09-26) Until the Quiet Comes (Album)
() 2010-06-18: Sonar 2010, Barcelona, SP (Live)


#### only albums

In [36]:
result1 = mbz.get_artist_by_id(artist_id,includes=["release-groups","tags"], release_type=["album"])

In [37]:
# get genres of albums
for release_group in result1["artist"]["release-group-list"]:
    if release_group['type'] == 'Album' and release_group['title'] != 'Lovers Melt 4':
        print("({date}) {tag} {title} ({type})".format(date=release_group["first-release-date"],
                                                tag=release_group["tag-list"],title=release_group["title"],type=release_group['type']))

(2008-06-04) [{'name': 'beatgen', 'count': '2'}, {'name': 'electronic', 'count': '3'}, {'name': 'hip-hop', 'count': '1'}, {'name': 'instrumental', 'count': '2'}] Los Angeles (Album)
(2006-10-03) [{'name': 'electronic', 'count': '1'}, {'name': 'hip-hop', 'count': '2'}, {'name': 'instrumental', 'count': '2'}] 1983 (Album)
(2014-10-06) [{'name': 'electronic', 'count': '1'}] You’re Dead! (Album)
(2010-04-21) [{'name': 'electronic', 'count': '2'}, {'name': 'hip-hop idm', 'count': '1'}, {'name': 'hip-hop industrial', 'count': '1'}] Cosmogramma (Album)
(2012-09-26) [{'name': 'electronic', 'count': '4'}] Until the Quiet Comes (Album)


In [38]:
newList = []
for record in result["artist"]["release-group-list"]:
    currentJson = {}
    currentJson['title'] = record.get('title')
    currentJson['date'] = record.get('first-release-date')
    currentJson['tag'] = record.get('tag-list')
    currentJson['type'] = record.get('type')
    newList.append(currentJson)

In [39]:
df = pd.DataFrame(newList)
df['year'] = df['date'].str[:4]

In [40]:
newdf = pd.DataFrame(df.groupby('year').type.value_counts())
newdf.columns = ['count']

In [41]:
temp1 = newdf.reset_index(0)
temp1['type'] = temp1.index

## Call Spotify API

In [42]:
import spotipy
sp = spotipy.Spotify()

In [43]:
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

In [44]:
get_artist("Flying Lotus")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/29XOeO6KIWxGthejQqn793'},
 'followers': {'href': None, 'total': 231603},
 'genres': ['wonky'],
 'href': 'https://api.spotify.com/v1/artists/29XOeO6KIWxGthejQqn793',
 'id': '29XOeO6KIWxGthejQqn793',
 'images': [{'height': 912,
   'url': 'https://i.scdn.co/image/83b48b244611bf21fe0112334c576f6a5e851664',
   'width': 912},
  {'height': 640,
   'url': 'https://i.scdn.co/image/34594e7abfe8fa44049fb944c59e479b4f431425',
   'width': 640},
  {'height': 200,
   'url': 'https://i.scdn.co/image/b6279ec650f4c8aa9a0e9ad4ff7f2090d5dd7d41',
   'width': 200},
  {'height': 64,
   'url': 'https://i.scdn.co/image/48e3ae8d3d021a3b797a4f0c1fc36cefa38f92b2',
   'width': 64}],
 'name': 'Flying Lotus',
 'popularity': 61,
 'type': 'artist',
 'uri': 'spotify:artist:29XOeO6KIWxGthejQqn793'}

In [45]:
uri = "spotify:artist:29XOeO6KIWxGthejQqn793"

In [46]:
related_artists = sp.artist_related_artists(uri)

In [47]:
def show_artist_albums(artist):
    albums = []
    results = sp.artist_albums(artist, album_type=['album'])
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    # to avoid dups
    seen = set()
    albums.sort(key=lambda album:album['name'].lower())
    for album in albums:
        name = album['name']
        uri = album['uri']
        if name not in seen:
            print((' ' + name)+''+(' ' + uri))
            seen.add(name)

In [48]:
show_artist_albums(uri)

 Cosmogramma spotify:album:5c7XChrHxYaqykCZLaGM5f
 Los Angeles spotify:album:7pdhnkO0t8HiRv4dmggt2I
 Until The Quiet Comes spotify:album:40aG9ahuLnAv96yoFG75Uy
 You're Dead! spotify:album:29luvT98TnqHjVDYSRbbrj
 You're Dead! (Deluxe) spotify:album:3Y9XeEhhPyzFnCGDjKcVB0


In [49]:
cosmo = 'spotify:album:5c7XChrHxYaqykCZLaGM5f'

In [55]:
album_tracks = sp.album_tracks(cosmo)

In [58]:
songid = '3C1z7AyGWhSNITaiz5tgws'

In [60]:
#audio = sp.audio_features(songid)